### Hashing 

A hash function is a function that takes an inpute and deterministically converts it to an integer that is less than a fized size set by the programmer. Inputs are called **keys** and the same input will always be converted to the same integer.

**Check if the sentence is Pangram**
- A pangram is a sentence where every letter of the English alphabet appears at least once.

In [ ]:
#Q: Given a string sentence containing only lowercase English letters, return true if sentence is a pangram or false otherwise
def is_pangram(sentence):
    sentence = len(sentence)
    seen = {}

    for i in sentence:
        seen.add(i)
        if len(seen) == 26:
            return True 
        
    return len(seen) == 26 